In [17]:
def prediction(model_path):
    # Import Library
    import pandas as pd
    import selenium
    import time
    from tqdm import tqdm
    import random as rd
    from datetime import datetime
    import tensorflow as tf

    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.action_chains import ActionChains
    import pyperclip

    from preprocessing.pre_for_df import preprocessing_df as pre_df
    from preprocessing.pre_for_train import preprocessing_train as pre_train
    
    def crawling():
        print('Crawling Today News..')
        sports_news = {'news':[]}

        driver = webdriver.Chrome()
        driver.get('https://www.naver.com/')
        driver.maximize_window()
        time.sleep(rd.uniform(0.3,1))

        driver.find_element(By.CSS_SELECTOR,r'#account > div > a').click()
        time.sleep(rd.uniform(0.3,1))

        # Log In
        # user_id = str(input('아이디를 입력하세요: '))
        # user_passwd = str(input('비밀번호를 입력하세요: '))
        user_id = 'sysconan'
        user_passwd = '!Brandan0211'

        driver.find_element(By.CSS_SELECTOR,r'#id_line').click()
        pyperclip.copy(user_id)
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
        time.sleep(rd.uniform(0.3,1))

        driver.find_element(By.CSS_SELECTOR,r'#pw').click()
        pyperclip.copy(user_passwd)
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
        time.sleep(rd.uniform(0.3,1))

        driver.find_element(By.CSS_SELECTOR,r'#log\.login').click()
        time.sleep(rd.uniform(2,2.5))

        # Go To Sports Page
        driver.execute_script("window.scrollTo(300, 0);")
        driver.find_element(By.CSS_SELECTOR,r'#shortcutArea > ul > li:nth-child(10) > a').click()
        driver.find_element(By.CSS_SELECTOR,r'#shortcutArea > div > ul > li:nth-child(9) > a').click()
        time.sleep(rd.uniform(0.3,1))
        driver.switch_to.window(driver.window_handles[0])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.find_element(By.CSS_SELECTOR,r'#lnb_list > li:nth-child(13) > a').click()
        time.sleep(rd.uniform(1,1.5))

        start = time.time()

        # Crawling Titles, Articles, Interested
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        news_lst = driver.find_elements(By.CSS_SELECTOR, 'div#_newsList > ul > li > div.text')
        date = driver.find_element(By.CSS_SELECTOR,'span.day').text[:10]
        for news in tqdm(range(len(news_lst))):
            try:
                driver.find_element(By.CSS_SELECTOR,
                                    f'div#_newsList > ul > li:nth-child({news+1}) > div.text > a').click()
                title = driver.find_element(By.CSS_SELECTOR,'h4.title').text
                driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
                article = driver.find_element(By.CSS_SELECTOR,'div#newsEndContents').text
                article = article.replace('\n','')

                # Delete Unnecessary Elements
                for div in driver.find_elements(By.CSS_SELECTOR,'div#newsEndContents > div'):
                    article = article.replace(div.text,'')

                for p in driver.find_elements(By.CSS_SELECTOR,'div#newsEndContents > p'):
                    article = article.replace(p.text,'')

                for em in driver.find_elements(By.CSS_SELECTOR,'div#newsEndContents > span > em'):
                    article = article.replace(em.text,'')

                for td in driver.find_elements(By.CSS_SELECTOR,
                                               'div#newsEndContents > font > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td'):
                    article = article.replace(td.text,'')

                if driver.find_element(By.CSS_SELECTOR,
                                       '#__markup_tomain > div > a').get_attribute('aria-pressed') == 'true':
                    interested = 1
                else:
                    interested = 0

                sports_news['news'].append({'date':date,
                                           'title':title,
                                           'article':article,
                                           'interested':interested,
                                           'link':str(driver.current_url)})

            except:
                pass

            driver.back()
            time.sleep(rd.uniform(0.3,0.5))

        print(f"time : {time.time() - start}s")
        driver.quit()
        
        df = pd.DataFrame(sports_news['news'])
        
        return df
    
    def preprocessing(df):
        for raw in range(len(df)):
            df.loc[raw,'date'] = df.loc[raw,'date'].replace('.','-')
            df.loc[raw,'date'] = datetime.strptime(df.loc[raw,'date'],'%Y-%m-%d')
            
        df1 = df.copy()
        df1 = pre_df(df1)
        x_pred, y_pred = pre_train(df1['title'],df1['interested'], split=False)
        
        return x_pred, y_pred
    
    def predicting(x_pred, model_path):
        model = tf.keras.models.load_model(model_path)
        score = model.predict(x_pred)
        score = score.astype('float')
        
        target = []
        for i in score:
            target.append(float(i))
            
    df = crawling()
    x_pred, y_pred = preprocessing(df)
    predicting(x_pred, model_path)
    
    df['interested'] = target
    
    return df

In [18]:
df = prediction('./best.h5')

Crawling Today News..


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:44<00:00,  2.21s/it]


time : 44.226731300354004s


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 58.81it/s]


1/1 [==============================] - 0s 269ms/step


In [20]:
df.head()

,date,title,article,interested,link
0,2023-06-13 00:00:00,"""맨유, 너무 느리게 행동한 대가 치를지도""…김민재 영입 못할 수도 있다 '英 매체...",맨체스터 유나이티드가 김민재를 영입하지 못할 수도 있다는 영국 현지 매체의 경고성 ...,0.016168,https://sports.news.naver.com/news?oid=413&aid...
1,2023-06-13 00:00:00,"""이강인이 ATM에 'PL 간다' 통보""…스페인 언론이 밝힌 '협상 결렬' 전말",(엑스포츠뉴스 김현기 기자) 이강인의 스페인 라리가 아틀레티코 마드리드 입단 협상이...,0.861595,https://sports.news.naver.com/news?oid=311&aid...
2,2023-06-13 00:00:00,"‘이게 무슨 X망신이야’…인종차별 논란, 연맹·구단 징계 떠나 국가대표 자격 없다",인종차별 논란을 일으킨 선수에게 국가대표가 될 자격은 없다.최근 울산 현대 선수들의...,0.005314,https://sports.news.naver.com/news?oid=410&aid...
3,2023-06-13 00:00:00,"""이 여권이 왜 안돼요?"" 메시 황당, 중국 공항서 2시간 붙잡혀",[스포츠조선 한동훈 기자] '축구의 신' 리오넬 메시가 중국 공항에서 약 2시간 동...,0.947676,https://sports.news.naver.com/news?oid=076&aid...
4,2023-06-13 00:00:00,"이강인 끝내 스페인 떠나 EPL 향하나.. ""ATM, 서프라이즈 아니면 영입 없다""",[OSEN=강필주 기자] 스페인 라리가 명문 아틀레티코 마드리드(ATM)가 이강인(...,0.440012,https://sports.news.naver.com/news?oid=109&aid...
